In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv("data/euro2024.csv")

In [3]:
df.head()

,Group,Team,Venue_1,Venue_2,Venue_3
0,A,2,Munich,Cologne,Stuttgart
1,A,3,Cologne,Stuttgart,Stuttgart
2,A,4,Cologne,Cologne,Frankfurt
3,B,1,Berlin,Gelsenkirchen,Düsseldorf
4,B,2,Berlin,Hamburg,Leipzig


In [4]:
df['GroupTeam'] = df['Group'].astype(str) + df['Team'].astype(str)

In [5]:
stad_dict  = {'Munich':[48.218889, 11.624722],
              'Cologne':[50.933497, 6.874997],
              'Stuttgart':[48.792269, 9.232031],
              'Frankfurt':[50.068056, 8.645806],
              'Berlin':[52.514722, 13.239444],
              'Gelsenkirchen':[51.554503, 7.067589],
              'Düsseldorf':[51.261667, 6.733056],
              'Dortmund':[51.492569, 7.451842],
              'Hamburg':[53.587159784503406, 9.898623812333906],
              'Leipzig':[51.34575, 12.348278]}

In [6]:
midpoints_lat = []
for i in df['GroupTeam']:
    lat = stad_dict[df['Venue_1'][df['GroupTeam']==i].item()][0] + stad_dict[df['Venue_2'][df['GroupTeam']==i].item()][0] + stad_dict[df['Venue_3'][df['GroupTeam']==i].item()][0]
    lat = lat/3
    midpoints_lat.append(lat)
df['midpoints_lat'] = midpoints_lat

midpoints_lon = []
for i in df['GroupTeam']:
    lon = stad_dict[df['Venue_1'][df['GroupTeam']==i].item()][1] + stad_dict[df['Venue_2'][df['GroupTeam']==i].item()][1] + stad_dict[df['Venue_3'][df['GroupTeam']==i].item()][1]
    lon = lon/3
    midpoints_lon.append(lon)
    
df['midpoints_lon'] = midpoints_lon

df

,Group,Team,Venue_1,Venue_2,Venue_3,GroupTeam,midpoints_lat,midpoints_lon
0,A,2,Munich,Cologne,Stuttgart,A2,49.314885,9.243917
1,A,3,Cologne,Stuttgart,Stuttgart,A3,49.506012,8.446353
2,A,4,Cologne,Cologne,Frankfurt,A4,50.645017,7.465267
3,B,1,Berlin,Gelsenkirchen,Düsseldorf,B1,51.776964,9.013363
4,B,2,Berlin,Hamburg,Leipzig,B2,52.482544,11.828782
5,B,3,Dortmund,Hamburg,Leipzig,B3,52.141826,9.899581
6,B,4,Dortmund,Hamburg,Düsseldorf,B4,52.113799,8.027841
7,C,1,Stuttgart,Munich,Cologne,C1,49.314885,9.243917
8,C,2,Stuttgart,Frankfurt,Munich,C2,49.026405,9.834186
9,C,3,Gelsenkirchen,Munich,Munich,C3,49.330760,10.105678


In [7]:
group_colour = {'A':'purple', 'B':'green','C':'blue', 'D':'black','E':'beige','F':'red'}

In [8]:
m = folium.Map([51.3158, 9.4979],zoom_start=6)
marker_cluster = MarkerCluster(control=False).add_to(m)

for index, row in df.iterrows():
    # print(row)
    location = row['midpoints_lat'], row['midpoints_lon']
    
    folium.Marker(
        location=location,
        popup = row['GroupTeam'],
        icon = folium.Icon(color = group_colour[row['Group']])
    ).add_to(m)

m

In [9]:
data = df[['midpoints_lat','midpoints_lon']]
kmeans = KMeans(n_clusters=2)
kmeans.fit(data)
df['cluster'] = kmeans.labels_
cluster_dict = {0 : 'red', 1 : 'green'}

m = folium.Map([51.3158, 9.4979],zoom_start=6)

for index, row in df.iterrows():
    # print(row)
    location = row['midpoints_lat'], row['midpoints_lon']
    names = [row['GroupTeam'], row['Venue_1'], row['Venue_2'], row['Venue_3']]
    nl = '\n'
    
    folium.Marker(
        location=location,
        popup = folium.Popup(f'{nl.join(names)}'),
        icon = folium.Icon(color = cluster_dict[row['cluster']])
    ).add_to(m)

m

In [23]:
m = folium.Map([51.3158, 9.4979],zoom_start=6)

hotel_1 = [50.009956200007494, 8.301352250040914]
hotel_2 = [52.387992757021124, 9.73338067901097]
cluster_dict = {0 : 'purple', 1 : 'green'}

fg1 = folium.FeatureGroup(name='Midpoints')
fg2 = folium.FeatureGroup(name='Hotel')
fg3 = folium.FeatureGroup(name='Stadiums')

for key, value in stad_dict.items():
    
    folium.Marker(
        location = value,
        popup = key,
        icon = folium.Icon(color='red',icon='futbol-o',prefix='fa')).add_to(fg3)
    
folium.Marker(
    location = hotel_1,
    popup = 'Hannover B&B Hotel',
    icon = folium.Icon(color = 'blue', icon ='bed',prefix='fa')).add_to(fg2)
    
folium.Marker(
    location = hotel_2,
    popup = 'Mainz-Kostheim',
    icon = folium.Icon(color = 'blue', icon ='bed',prefix='fa')).add_to(fg2)

for index, row in df.iterrows():
    # print(row)
    location = row['midpoints_lat'], row['midpoints_lon']
    names = [row['GroupTeam'], row['Venue_1'], row['Venue_2'], row['Venue_3']]
    nl = '\n'
    
    folium.Marker(
        location=location,
        popup = folium.Popup(f'{nl.join(names)}'),
        icon = folium.Icon(icon='times',prefix='fa',color = cluster_dict[row['cluster']])
    ).add_to(fg1)

fg1.add_to(m)
fg2.add_to(m)
fg3.add_to(m)
folium.LayerControl().add_to(m)
m